# ВЕБ-ЗАПРОСЫ
Сначала разберемся как получить информацию с внешнего сервиса. Для этого есть много библиотек, самой распространенной является requests. Импортируем ее и отправим запрос к сервису:

In [1]:
import requests

r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')

In [2]:
print(r)

<Response [200]>


Мы получили объект ответа, который содержит всю нужную нам информацию. По умолчанию на экран выводится HTTP-код ответа 200. Это означает, что запрос был корректным и сервер отдал нам нужную информацию. Более подробную информацию о кодах состояния HTTP можно прочитать по ссылке.

Код ответа в виде числа можно получить с помощью метода status_code:

In [3]:
print(r.status_code)

200


# СОДЕРЖИМОЕ ОТВЕТА
Нужная нам информация содержится в тексте ответа сервера. Можно получить ее с помощью метода text:

In [4]:
print(r.text)

{
    "Date": "2018-12-07T11:30:00+03:00",
    "PreviousDate": "2018-12-06T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2018\/12\/06\/daily_json.js",
    "Timestamp": "2018-12-06T21:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 48.2247,
            "Previous": 48.7416
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 39.3385,
            "Previous": 39.3894
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 84.8036,
            "Previous": 84.9469
   

Однако, работать с ответом сервера в виде строки крайне неудобно. Гораздо проще вызвать метод json(), который сразу преобразует строку в словарь:

In [5]:
data = r.json()

In [6]:
data

{'Date': '2018-12-07T11:30:00+03:00',
 'PreviousDate': '2018-12-06T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2018/12/06/daily_json.js',
 'Timestamp': '2018-12-06T21:00:00+03:00',
 'Valute': {'AUD': {'ID': 'R01010',
   'NumCode': '036',
   'CharCode': 'AUD',
   'Nominal': 1,
   'Name': 'Австралийский доллар',
   'Value': 48.2247,
   'Previous': 48.7416},
  'AZN': {'ID': 'R01020A',
   'NumCode': '944',
   'CharCode': 'AZN',
   'Nominal': 1,
   'Name': 'Азербайджанский манат',
   'Value': 39.3385,
   'Previous': 39.3894},
  'GBP': {'ID': 'R01035',
   'NumCode': '826',
   'CharCode': 'GBP',
   'Nominal': 1,
   'Name': 'Фунт стерлингов Соединенного королевства',
   'Value': 84.8036,
   'Previous': 84.9469},
  'AMD': {'ID': 'R01060',
   'NumCode': '051',
   'CharCode': 'AMD',
   'Nominal': 100,
   'Name': 'Армянских драмов',
   'Value': 13.7717,
   'Previous': 13.7896},
  'BYN': {'ID': 'R01090B',
   'NumCode': '933',
   'CharCode': 'BYN',
   'Nominal': 1,
   'Name': 'Б

# ИЗВЛЕКАЕМ НУЖНЫЕ ДАННЫЕ
Теперь очень просто извлечь данные о курсах валют из ответа сервера. Для этого достаточно обратиться к ключу словаря 'Valute':

In [9]:
data['Valute']['INR']['ID']

'R01270'

# ОФОРМЛЯЕМ ФУНКЦИЮ
Давайте оформим наши вычисления в отдельную фукнцию, которой будет удобно пользоваться. На вход она должна принимать два параметра:

1. Название валюты currency. Например, 'EUR' или 'USD'.

2. Формат ответа format. При значении 'full' будем отдавать все что знаем о валюте. Например, для currency = 'USD':
```python
{'CharCode': 'USD',

   'ID': 'R01235',

   'Name': 'Доллар США',

   'Nominal': 1,

   'NumCode': '840',

   'Previous': 68.2505,

   'Value': 69.0286}
```

А при значении format = 'value' только значение ключа 'Value', т. е. курс: 69.0286.

Оформим наши требования в коде:

In [10]:
def exchange_rates(currency, format='full'):

    response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']
    data = response[currency]    

    if format == 'full':
        return data    

    elif format == 'value':
        return data['Value']

In [12]:
exchange_rates('USD', 'value')

66.7377

# НЕКОРРЕКТНАЯ ВАЛЮТА
Перед тем как потестировать нашу функцию добавим еще одну проверку: если пользователь ввел несуществующую валюту, то надо вернуть предупреждение. Аналогичную проверку добавим и для формата:

In [13]:
def exchange_rates(currency, format='full'):

    response = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']
    
    if currency in response:
        data = response[currency]   

    else:
        return 'Unknown currency!'    

    if format == 'full':
        return data    

    elif format == 'value':
        return data['Value']    

    else:
        return 'Unknown format!'

Теперь проверим корректность работы:

1. Валюта корректна, формат  'full':

In [14]:
exchange_rates(currency='USD')

{'ID': 'R01235',
 'NumCode': '840',
 'CharCode': 'USD',
 'Nominal': 1,
 'Name': 'Доллар США',
 'Value': 66.7377,
 'Previous': 66.8242}

In [15]:
exchange_rates('EUR', 'value')

75.6071

In [16]:
exchange_rates('EUR/USD')

'Unknown currency!'

In [17]:
exchange_rates('EUR', '12345')

'Unknown format!'

# Проверочное задание
(1 балл из 1)
Проверочное задание

Напишите функцию, которая по ID валюты возвращает ее название на русском языке. Например, для кода 'R01700J' это 'Турецкая лира'.

Какое значение должна вернуть функция для ID валюты 'R01565'?

In [28]:
def currency_name(valute_id):
    r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()['Valute']
    for key in r:
        if r[key]['ID'] == valute_id:
            return r[key]['Name']
    return 'Valute was not found'

In [31]:
currency_name('R01565')

'Польский злотый'